In [1]:
import subprocess
from matplotlib import pyplot as plt
%matplotlib inline
import pytesseract
from PIL import Image

In [2]:
TEXT_IMAGE = "ideal_version.jpg"
PHOTO_IMAGE = "new0.jpg"
PROCESSED_IMAGE = "new2.jpg"

In [3]:
def binarize(input_file_path, output_file_path="binarized_image.jpg"):
    process = subprocess.run(['./binarizewolfjolion', input_file_path, output_file_path])
    return Image.open(output_file_path)

In [4]:
def image_to_text(image_path, binarization_needed=True):
    if binarization_needed:
        image = binarize(image_path)
    else:
        image = Image.open(image_path)
    text = pytesseract.image_to_string(image)
    return text

Можно применять разные меры схожести двух текстов:

https://stackabuse.com/levenshtein-distance-and-text-similarity-in-python/

Остановимся на Cходствe Джаро—Винклера.

In [5]:
import jellyfish

In [6]:
def get_texts_similarity(original_image_path, image_path, binarization_needed=True):
    original_text = image_to_text(original_image_path, binarization_needed=binarization_needed)
    print(original_text, "\n\n\n")
    text = image_to_text(image_path, binarization_needed=binarization_needed)
    print(text)
    return jellyfish.jaro_winkler(original_text, text) 

In [7]:
get_texts_similarity(TEXT_IMAGE, PHOTO_IMAGE, binarization_needed=False)

DANDELION WINE

It was a quiet morning, the town covered over with darkness and at ease in bed. Summer gathered in the
weather, the wind had the proper touch, the breathing of the world was long and warm and slow. You had only to
rise, lean from your window, and know that this indeed was the first real time of freedom and living, this was the
first morning of summer.

Douglas Spaulding, twelve, freshly wakened, let summer idle him on its early-morning stream. Lying in his third-
story cupola bedroom, he felt the tall power it gave him, riding high in the June wind, the grandest tower in town. At
night, when the trees washed together, he flashed his gaze like a beacon from this lighthouse in all directions over
swarming seas of elm and oak and maple. Now . . .

“Boy,” whispered Douglas.

A whole summer ahead to cross off the calendar, day by day. Like the goddess Siva in the travel books, he saw his
hands jump everywhere, pluck sour apples, peaches, and midnight plums. He would be cloth

0.0

In [9]:
get_texts_similarity(TEXT_IMAGE, PROCESSED_IMAGE, binarization_needed=True)

DANDELION WINE

It was a quiet morning, the town covered over with darkness and at ease in bed. Summer gathered in the
weather, the wind had the proper touch, the breathing of the world was long and warm and slow. You had only to
rise, lean from your window, and know that this indeed was the first real time of freedom and living, this was the
first morning of summer.

Douglas Spaulding, twelve, freshly wakened, let summer idle him on its early-morning stream. Lying in his third-
story cupola bedroom, he felt the tall power it gave him, riding high in the June wind, the grandest tower in town, At
night, when the trees washed together, he flashed his gaze like a beacon from this lighthouse in all directions over
swarming seas of elm and oak and maple. Now . ..

“Boy,” whispered Douglas.

A whole summer ahead to cross off the calendar, day by day. Like the goddess Siva in the travel books, he saw his
hands jump everywhere, pluck sour apples, peaches, and midnight plums. He would be clothe

1.0